## Подготовка и установка пакетов

In [27]:
#For beauty, we remove warnings
import warnings
warnings.filterwarnings('ignore')

In [28]:
#Importing packages
import pandas as pd
from statsmodels.formula.api import ols

#Determining paths
input_path = "C:/Users/Дмитрий/Desktop/HW 3/Dataset_HA-3.xlsx"
output_path = "C:/Users/Дмитрий/Desktop/HW 3/Output_HA-3.xlsx"

## Задание 1. Разделение выборки на тренировочную и тестовую части.

In [29]:
#read data from excel file
df=pd.read_excel(input_path, sheet_name = "Data")

#date to split training and testing datasets
split_date=pd.datetime(2020,3,13)

#training dataset
df_train = df.loc[df['date']<=split_date]


## Задание 2. Оценка beta для разных моделей.

Описательная статистика внутри Excel файла на отдельном листе

In [30]:
#CAPM
c_intercept_capm = []
c_beta_capm = []
p_intercept_capm = []
p_beta_capm = []

#running 230 regressions (w.r.t. 230 companies)
for i in range(1,231):
    capm_train = ols('r'+str(i)+'~mrp',df_train).fit(cov_type='HAC',cov_kwds={'maxlags':12})
    c_intercept_capm.append(capm_train.params[0])
    c_beta_capm.append(capm_train.params[1])
    p_intercept_capm.append(capm_train.pvalues[0])
    p_beta_capm.append(capm_train.pvalues[1])


#creating dataframe to save in excel   
df_results_capm = pd.DataFrame({'c_intercept_capm':c_intercept_capm,'c_beta_capm':c_beta_capm,
                                'p_intercept_capm':p_intercept_capm,'p_beta_capm':p_beta_capm})

In [31]:
#Fama-French three factor model
c_intercept_FF = []
c_beta_FF = []
c_smb_FF = []
c_hml_FF = []
p_intercept_FF = []
p_beta_FF = []
p_smb_FF = []
p_hml_FF = []

#running 230 regressions (w.r.t. 230 companies)
for i in range(1,231):
    FF_train = ols('r'+str(i)+'~mrp+smb+hml',df_train).fit(cov_type='HAC',cov_kwds={'maxlags':12})
    c_intercept_FF.append(FF_train.params[0])
    c_beta_FF.append(FF_train.params[1])
    c_smb_FF.append(FF_train.params[2])
    c_hml_FF.append(FF_train.params[3])
    p_intercept_FF.append(FF_train.pvalues[0])
    p_beta_FF.append(FF_train.pvalues[1])
    p_smb_FF.append(FF_train.pvalues[2])
    p_hml_FF.append(FF_train.pvalues[3])


#creating dataframe to save in excel   
df_results_FF = pd.DataFrame({'c_intercept_FF':c_intercept_FF,'c_beta_FF':c_beta_FF, 
                              'c_smb_FF':c_smb_FF,'c_hml_FF':c_hml_FF,
                                'p_intercept_FF':p_intercept_FF,'p_beta_FF':p_beta_FF,
                              'p_smb_FF':p_smb_FF,'p_hml_FF':p_hml_FF})

In [32]:
#Fama-French-Carhart four factor model
c_intercept_FFC = []
c_beta_FFC = []
c_smb_FFC = []
c_hml_FFC = []
c_mom_FFC = []
p_intercept_FFC = []
p_beta_FFC = []
p_smb_FFC = []
p_hml_FFC = []
p_mom_FFC = []

#running 230 regressions (w.r.t. 230 companies)
for i in range(1,231):
    FFC_train = ols('r'+str(i)+'~mrp+smb+hml+mom',df_train).fit(cov_type='HAC',cov_kwds={'maxlags':12})
    c_intercept_FFC.append(FFC_train.params[0])
    c_beta_FFC.append(FFC_train.params[1])
    c_smb_FFC.append(FFC_train.params[2])
    c_hml_FFC.append(FFC_train.params[3])
    c_mom_FFC.append(FFC_train.params[4])
    p_intercept_FFC.append(FFC_train.pvalues[0])
    p_beta_FFC.append(FFC_train.pvalues[1])
    p_smb_FFC.append(FFC_train.pvalues[2])
    p_hml_FFC.append(FFC_train.pvalues[3])
    p_mom_FFC.append(FFC_train.pvalues[4])

#creating dataframe to save in excel   
df_results_FFC = pd.DataFrame({'c_intercept_FFC':c_intercept_FFC,'c_beta_FFC':c_beta_FFC, 
                              'c_smb_FFC':c_smb_FFC,'c_hml_FFC':c_hml_FFC, 'c_mom_FFC':c_mom_FFC,
                                'p_intercept_FFC':p_intercept_FFC, 'p_beta_FFC':p_beta_FFC,
                              'p_smb_FFC':p_smb_FFC,'p_hml_FFC':p_hml_FFC, 'p_mom_FFC':p_mom_FFC})

In [33]:
#saving dataframe to excel
output=pd.ExcelWriter(output_path,engine='xlsxwriter')
df_results_capm.to_excel(output,index=False,sheet_name='results_capm')
df_results_FF.to_excel(output,index=False,sheet_name='results_FF')
df_results_FFC.to_excel(output,index=False,sheet_name='results_FFC')
output.save()

## Задание 3. Регрессии на тестовой выборке

#### Общий комментарий по качеству моделей:
По результатам оценки трех моделей получаем следующие значения R-квадрат (adjusted):

|Модель | Первый тестовый день | Второй тестовый день | Третий тестовый день |
| ------- | -------- | ------- | -------- |
|CAPM | 4,5% | 2,6% | 6,2% |
|Fama-French | 34,4% | 23,2% | 19,7% |
|Fama-French-Carhart | 35,4% | 23,2% | 20,2% |

Если сравнивать модели по R-квадрат (adj), то мы видим, что использование трех или четырехфакторных моделей значительно повышает его значения - с 3-6% до 20-30%. Одновременно с этим, мы видим, что модель CAPM объясняет всего 3-6% от изменения цен активов, что очень мало.



#### Комментарий по значимости факторов:
При расширении классической CAPM-модели до трехфакторной или четырехфакторной коэффициент при beta становится незначим. При расширении классической модели Фама-Френча до четырехфакторной коэффициент при переменной моментума оказазывается незначимым, что также видно из практически не меняющегося R-квадрат (adj).

##### Значимость факторов в CAPM (на 5%-м уровне):
|Факторы | Первый тестовый день | Второй тестовый день | Третий тестовый день |
| ------- | -------- | ------- | -------- |
|Const | да | да | да |
|Beta | да | да | да |

##### Значимость факторов в Fama-French (на 5%-м уровне):
|Факторы | Первый тестовый день | Второй тестовый день | Третий тестовый день |
| ------- | -------- | ------- | -------- |
|Const | да | **нет** | да |
|Beta | **нет** | **нет** | **нет** |
|SMB | да | да | да |
|HML | да | да | **нет** |

##### Значимость факторов в Fama-French-Carhart (на 5%-м уровне):
|Факторы | Первый тестовый день | Второй тестовый день | Третий тестовый день |
| ------- | -------- | ------- | -------- |
|Const | да | **нет** | да |
|Beta | **нет** | **нет** | **нет** |
|SMB | да | да | да |
|HML | да | да | **нет** |
|MOM | **нет** | **нет** | **нет** |

In [34]:
#CAPM models for testing days
r_one_capm=[]
r_two_capm=[]
r_three_capm=[]

#creating vectors with data of 16.03.2020-18.03.2020 - returns of 230 companies
for i in range(1,231):
    r_one_capm.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,16)].values))
    r_two_capm.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,17)].values))
    r_three_capm.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,18)].values))
    
    
#creating testing dataset
df_test_capm=pd.DataFrame({'r_one_capm':r_one_capm,'r_two_capm':r_two_capm,'r_three_capm':r_three_capm,'c_beta_capm':c_beta_capm})

#Fama-McBeth regression
capm_test_one = ols('r_one_capm~c_beta_capm',df_test_capm).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(capm_test_one.summary())

capm_test_two = ols('r_two_capm~c_beta_capm',df_test_capm).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(capm_test_two.summary())

capm_test_three = ols('r_three_capm~c_beta_capm',df_test_capm).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(capm_test_three.summary())

                            OLS Regression Results                            
Dep. Variable:             r_one_capm   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     17.43
Date:                Sun, 26 Feb 2023   Prob (F-statistic):           4.25e-05
Time:                        19:04:03   Log-Likelihood:                 320.38
No. Observations:                 230   AIC:                            -636.8
Df Residuals:                     228   BIC:                            -629.9
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0395      0.009     -4.444      

In [35]:
#Fama-French models for testing days
r_one_FF=[]
r_two_FF=[]
r_three_FF=[]

#creating vectors with data of 16.03.2020-18.03.2020 - returns of 230 companies
for i in range(1,231):
    r_one_FF.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,16)].values))
    r_two_FF.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,17)].values))
    r_three_FF.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,18)].values))
    

#creating testing dataset
df_test_FF=pd.DataFrame({'r_one_FF':r_one_FF,'r_two_FF':r_two_FF,'r_three_FF':r_three_FF,
                         'c_beta_FF':c_beta_FF,'c_smb_FF':c_smb_FF,'c_hml_FF':c_hml_FF})

#Fama-McBeth regression
FF_test_one = ols('r_one_FF~c_beta_FF+c_smb_FF+c_hml_FF',df_test_FF).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(FF_test_one.summary())

FF_test_two = ols('r_two_FF~c_beta_FF+c_smb_FF+c_hml_FF',df_test_FF).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(FF_test_two.summary())

FF_test_three = ols('r_three_FF~c_beta_FF+c_smb_FF+c_hml_FF',df_test_FF).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(FF_test_three.summary())

                            OLS Regression Results                            
Dep. Variable:               r_one_FF   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     35.40
Date:                Sun, 26 Feb 2023   Prob (F-statistic):           8.60e-19
Time:                        19:04:03   Log-Likelihood:                 364.53
No. Observations:                 230   AIC:                            -721.1
Df Residuals:                     226   BIC:                            -707.3
Df Model:                           3                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0675      0.010     -6.792      0.0

In [36]:
#Fama-French-Carhart models for testing days
r_one_FFC=[]
r_two_FFC=[]
r_three_FFC=[]

#creating vectors with data of 16.03.2020-18.03.2020 - returns of 230 companies
for i in range(1,231):
    r_one_FFC.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,16)].values))
    r_two_FFC.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,17)].values))
    r_three_FFC.append(float(df['r'+str(i)][df['date']==pd.datetime(2020,3,18)].values))
    

#creating testing dataset
df_test_FFC=pd.DataFrame({'r_one_FFC':r_one_FFC,'r_two_FFC':r_two_FFC,'r_three_FFC':r_three_FFC,
                         'c_beta_FFC':c_beta_FFC,'c_smb_FFC':c_smb_FFC,'c_hml_FFC':c_hml_FFC,'c_mom_FFC':c_mom_FFC})

#Fama-McBeth regression
FFC_test_one = ols('r_one_FFC~c_beta_FFC+c_smb_FFC+c_hml_FFC+c_mom_FFC',df_test_FFC).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(FFC_test_one.summary())

FFC_test_two = ols('r_two_FFC~c_beta_FFC+c_smb_FFC+c_hml_FFC+c_mom_FFC',df_test_FFC).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(FFC_test_two.summary())

FFC_test_three = ols('r_three_FFC~c_beta_FFC+c_smb_FFC+c_hml_FFC+c_mom_FFC',df_test_FFC).fit(cov_type='HAC',cov_kwds={'maxlags':0})
print(FFC_test_three.summary())

                            OLS Regression Results                            
Dep. Variable:              r_one_FFC   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     30.91
Date:                Sun, 26 Feb 2023   Prob (F-statistic):           1.65e-20
Time:                        19:04:03   Log-Likelihood:                 366.87
No. Observations:                 230   AIC:                            -723.7
Df Residuals:                     225   BIC:                            -706.5
Df Model:                           4                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0641      0.010     -6.460      0.0